In [90]:
import torch 
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

![yyPd9I](https://gitee.com/pxqp9W/testmarkdown/raw/master/imgs/2020/07/yyPd9I.png)

In [91]:
class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32) 
        self.len = xy.shape[0] # xy.shape输出矩阵的形状，取[0]即可取出长度
        self.x_data = torch.from_numpy(xy[:, :-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        return self.len

In [92]:
dataset = DiabetesDataset('./dataset/diabetes.csv.gz') 
train_loader = DataLoader(dataset=dataset,batch_size=32, shuffle=True, num_workers=2)

In [93]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__() 
        self.linear1 = torch.nn.Linear(8, 6) 
        self.linear2 = torch.nn.Linear(6, 4) 
        self.linear3 = torch.nn.Linear(4, 1) 
        self.sigmoid = torch.nn.Sigmoid() # 激活函数也可以选择其他的
    def forward(self, x):
        x = self.sigmoid(self.linear1(x)) 
        x = self.sigmoid(self.linear2(x)) 
        x = self.sigmoid(self.linear3(x)) 
        return x

In [94]:
model = Model()

In [95]:
# Construct loss and optimizer using PyTorch API
criterion = torch.nn.BCELoss(size_average=False) # Logistic Regression使用BCE作为损失函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [96]:
# traing cycle
for epoch in range(100):
    for i, data in enumerate(train_loader, 0): # i是索引，data是从train_loader中取出的数据
        # 1. Prepare data
        inputs,labels = data # inputs和labels都是tensor
        # 2. Forward
        y_pred = model(inputs) 
        loss = criterion(y_pred, labels) 
        print("epoch: " ,epoch, "\t iteration: ",i, " loss: ",loss.item())
        # 3. Backward
        optimizer.zero_grad() 
        loss.backward()
        # 4. Update
        optimizer.step()

epoch:  0 	 iteration:  0  loss:  23.182767868041992
epoch:  0 	 iteration:  1  loss:  18.162187576293945
epoch:  0 	 iteration:  2  loss:  19.037464141845703
epoch:  0 	 iteration:  3  loss:  20.9598388671875
epoch:  0 	 iteration:  4  loss:  24.314250946044922
epoch:  0 	 iteration:  5  loss:  21.360008239746094
epoch:  0 	 iteration:  6  loss:  20.590648651123047
epoch:  0 	 iteration:  7  loss:  22.504924774169922
epoch:  0 	 iteration:  8  loss:  20.59164047241211
epoch:  0 	 iteration:  9  loss:  22.620534896850586
epoch:  0 	 iteration:  10  loss:  19.897689819335938
epoch:  0 	 iteration:  11  loss:  19.027469635009766
epoch:  0 	 iteration:  12  loss:  21.75818634033203
epoch:  0 	 iteration:  13  loss:  21.85863494873047
epoch:  0 	 iteration:  14  loss:  22.05298614501953
epoch:  0 	 iteration:  15  loss:  21.64790916442871
epoch:  0 	 iteration:  16  loss:  20.182374954223633
epoch:  0 	 iteration:  17  loss:  25.31786346435547
epoch:  0 	 iteration:  18  loss:  21.87298965

一个batch_size是32，CSV文件一共758行，故每一个epoch中有24次iteration